# 1.导入所需库 
①Pyproj库：用于进行地理投影坐标系转换；②Xarray：数组相关；③Numpy：用于处理大型矩阵；④zarr：适用于Python的分块,压缩N维数组的实现；

In [4]:
from pyproj import Proj, transform
from math import radians, cos
from io import StringIO
import xarray as xr   
import pandas as pd
import numpy as np
import datetime
import sys
import os
import zarr
import cftime

import warnings
warnings.filterwarnings('ignore')
np.set_printoptions(suppress=True)

# 2.导入matplotli，用于绘图
使用 %matplotlib inline 命令，可以将matplotlib的图表直接嵌入到Notebook之中

In [5]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib
matplotlib.rc('font', **{'size': 16})

In [6]:
ds = xr.open_dataset("data/MCD43A1.nc")
ds = ds.rename({"Num_Parameters": "param", "xdim": "x", "ydim": "y"})

ds

<xarray.Dataset>
Dimensions:                                       (time: 10, y: 3045, x: 3265,
                                                   param: 3)
Coordinates:
  * time                                          (time) object 2005-01-01 00...
  * y                                             (y) float64 4.082e+06 ... 2...
  * x                                             (x) float64 6.265e+06 ... 7...
  * param                                         (param) int32 0 1 2
Data variables: (12/21)
    crs                                           int8 ...
    BRDF_Albedo_Band_Mandatory_Quality_Band1      (time, y, x) float32 ...
    BRDF_Albedo_Band_Mandatory_Quality_Band2      (time, y, x) float32 ...
    BRDF_Albedo_Band_Mandatory_Quality_Band3      (time, y, x) float32 ...
    BRDF_Albedo_Band_Mandatory_Quality_Band4      (time, y, x) float32 ...
    BRDF_Albedo_Band_Mandatory_Quality_Band5      (time, y, x) float32 ...
    ...                                            ...
    BRDF_Albedo_Parameters_Band5                  (time, y, x, param) float32 ...
    BRDF_Albedo_Parameters_Band6                  (time, y, x, param) float32 ...
    BRDF_Albedo_Parameters_Band7                  (time, y, x, param) float32 ...
    BRDF_Albedo_Parameters_nir                    (time, y, x, param) float32 ...
    BRDF_Albedo_Parameters_shortwave              (time, y, x, param) float32 ...
    BRDF_Albedo_Parameters_vis                    (time, y, x, param) float32 ...
Attributes:
    title:        MCD43A1.006 for aid0001
    Conventions:  CF-1.6
    institution:  Land Processes Distributed Active Archive Center (LP DAAC)
    source:       AppEEARS v3.2
    references:   See README.md
    history:      See README.md

# 3.测试代码
发现数据沿着三维：time、x、y 组织起来

In [13]:
ds = ds.isel(time=slice(0,10))
ds

<xarray.Dataset>
Dimensions:                                       (time: 9, y: 3045, x: 3265,
                                                   param: 3)
Coordinates:
  * time                                          (time) object 2005-01-01 00...
  * y                                             (y) float64 4.082e+06 ... 2...
  * x                                             (x) float64 6.265e+06 ... 7...
  * param                                         (param) int32 0 1 2
Data variables: (12/21)
    crs                                           int8 -127
    BRDF_Albedo_Band_Mandatory_Quality_Band1      (time, y, x) float32 ...
    BRDF_Albedo_Band_Mandatory_Quality_Band2      (time, y, x) float32 ...
    BRDF_Albedo_Band_Mandatory_Quality_Band3      (time, y, x) float32 ...
    BRDF_Albedo_Band_Mandatory_Quality_Band4      (time, y, x) float32 ...
    BRDF_Albedo_Band_Mandatory_Quality_Band5      (time, y, x) float32 ...
    ...                                            ...
    BRDF_Albedo_Parameters_Band5                  (time, y, x, param) float32 ...
    BRDF_Albedo_Parameters_Band6                  (time, y, x, param) float32 ...
    BRDF_Albedo_Parameters_Band7                  (time, y, x, param) float32 ...
    BRDF_Albedo_Parameters_nir                    (time, y, x, param) float32 ...
    BRDF_Albedo_Parameters_shortwave              (time, y, x, param) float32 ...
    BRDF_Albedo_Parameters_vis                    (time, y, x, param) float32 ...
Attributes:
    title:        MCD43A1.006 for aid0001
    Conventions:  CF-1.6
    institution:  Land Processes Distributed Active Archive Center (LP DAAC)
    source:       AppEEARS v3.2
    references:   See README.md
    history:      See README.md

# 4.坐标和太阳天顶角
①添加lat和lon值的二维数组
②将一些关于质量标志的信息和元数据，分配给Mandatory_Quality变量等

需要纬度和经度来计算太阳天顶角，所以使用numpy和proj库为数据集中的x，y坐标生成lat，lon等效数组。
需要采取以下几个步骤：

# （1）定义SRS
使用pyproj定义源x、y阵列和目标lon、lat阵列的空间信息。

In [15]:
ds.crs

<xarray.DataArray 'crs' ()>
array(-127, dtype=int8)
Attributes:
    grid_mapping_name:                      sinusoidal
    _CoordinateAxisTypes:                   GeoX GeoY
    spatial_ref:                            PROJCS["unknown",GEOGCS["unknown"...
    radius_of_sphere:                       6371007.181
    longitude_of_central_meridian:          0
    longitude_of_projection_origin:         0
    straight_vertical_longitude_from_pole:  0
    false_easting:                          0
    false_northing:                         0

# 使用WGS84的EPSG代码：
错误未解决

In [18]:
getpar = lambda a: str(ds.crs.attrs[a])

inproj = Proj(" ".join([
    "+proj=sinu",
    "+lon_0="+getpar("longitude_of_central_meridian"),
    "+x_0="+getpar("false_easting"),
    "+y_0="+getpar("false_northing"),
    "+a="+getpar("semi_major_axis"),
    "+b="+getpar("semi_minor_axis"),
    "+units="+"meter +no_defs"]))

outproj = Proj(init="epsg:4326")

inproj

KeyError: 'semi_major_axis'